In [69]:
import pandas as pd 
import numpy as np

excel_file = pd.ExcelFile("./data/gmo_data.xlsx")

sheet_names = excel_file.sheet_names
data = pd.read_excel("./data/gmo_data.xlsx", sheet_name="total returns")
rf = pd.read_excel("./data/gmo_data.xlsx", sheet_name="risk-free rate")
signals = pd.read_excel("./data/gmo_data.xlsx", sheet_name="signals")

data.head()

,date,SPY,GMWAX,GMGEX
0,1996-12-31,-0.023292,-0.022094,-0.013000
1,1997-01-31,0.061786,0.014735,0.034448
2,1997-02-28,0.009565,0.022265,0.012733
3,1997-03-31,-0.045721,-0.015152,-0.016441
4,1997-04-30,0.064368,-0.006731,0.000000


In [76]:
signals.set_index('date', inplace=True)

In [70]:
data = data.set_index('date')
rf = rf.set_index('date')
excess_returns = data.subtract(rf['TBill 3M'], axis=0)
excess_returns

,SPY,GMWAX,GMGEX
date,,,
1996-12-31,-0.075002,-0.073804,-0.064710
1997-01-31,0.010316,-0.036735,-0.017022
1997-02-28,-0.042635,-0.029935,-0.039467
1997-03-31,-0.098941,-0.068372,-0.069661
1997-04-30,0.012038,-0.059061,-0.052330
...,...,...,...
2024-06-28,-0.018264,-0.060741,-0.066632
2024-07-31,-0.040731,-0.022440,-0.018159
2024-08-30,-0.027751,-0.036423,-0.031575


In [60]:
annualization_factor = 12
def performance_summary(return_data, annualization_factor):
    """ 
    Returns the Performance Stats for given set of returns
    Inputs: 
        return_data - DataFrame or Series with Date index and Monthly Returns for different assets/strategies.
        Output:
            summary_stats - DataFrame with annualized mean return, vol, Sharpe ratio, Skewness, 
                            Excess Kurtosis, VaR (0.05), CVaR (0.05), and drawdown based on monthly returns. 
    """
    # Ensure input is a DataFrame for consistent handling
    if isinstance(return_data, pd.Series):
        return_data = return_data.to_frame(name='Return')
    
    summary_stats = pd.DataFrame()

    # Compute metrics
    summary_stats['Mean'] = return_data.mean() * annualization_factor
    summary_stats['Volatility'] = return_data.std() * np.sqrt(annualization_factor)
    summary_stats['Sharpe Ratio'] = summary_stats['Mean'] / summary_stats['Volatility']

    summary_stats['Skewness'] = return_data.skew()
    summary_stats['Excess Kurtosis'] = return_data.kurtosis()
    summary_stats['VaR (0.05)'] = return_data.quantile(0.05)
    summary_stats['CVaR (0.05)'] = return_data[return_data.le(return_data.quantile(0.05))].mean()
    summary_stats['Min'] = return_data.min()
    summary_stats['Max'] = return_data.max()

    # Calculate drawdowns
    wealth_index = 1000 * (1 + return_data).cumprod()
    previous_peaks = wealth_index.cummax()
    drawdowns = (wealth_index - previous_peaks) / previous_peaks

    summary_stats['Max Drawdown'] = drawdowns.min()

    # Handling peak, bottom, and recovery for Series or DataFrame
    if isinstance(return_data, pd.DataFrame):
        summary_stats['Peak'] = [
            previous_peaks[col][:drawdowns[col].idxmin()].idxmax() for col in previous_peaks.columns
        ]
        summary_stats['Bottom'] = drawdowns.idxmin()
        
        recovery_date = []
        for col in wealth_index.columns:
            prev_max = previous_peaks[col][:drawdowns[col].idxmin()].max()
            recovery_wealth = wealth_index[col][drawdowns[col].idxmin():]
            recovery_date.append(recovery_wealth[recovery_wealth >= prev_max].index.min())
        summary_stats['Recovery'] = recovery_date
    else:
        col = 'Return'
        summary_stats['Peak'] = previous_peaks[:drawdowns.idxmin()].idxmax()
        summary_stats['Bottom'] = drawdowns.idxmin()
        
        prev_max = previous_peaks[:drawdowns.idxmin()].max()
        recovery_wealth = wealth_index[drawdowns.idxmin():]
        summary_stats['Recovery'] = recovery_wealth[recovery_wealth >= prev_max].index.min()

    return summary_stats


In [82]:
def time_series_regression(portfolio, factors, annualization_factor, multiple_factors = False, resid = False):
    
    ff_report = pd.DataFrame(index=portfolio.columns)
    bm_residuals = pd.DataFrame(columns=portfolio.columns)

    rhs = sm.add_constant(factors)

    for portf in portfolio.columns:
        lhs = portfolio[portf]
        res = sm.OLS(lhs, rhs, missing='drop').fit()
        ff_report.loc[portf, 'alpha_hat'] = res.params['const'] * annualization_factor
        if multiple_factors:
            ff_report.loc[portf, factors.columns[0] + ' beta'] = res.params[1]
            ff_report.loc[portf, factors.columns[1]+ ' beta'] = res.params[2] 
            ff_report.loc[portf, factors.columns[2]+ ' beta'] = res.params[3]
        else:
            ff_report.loc[portf, factors.columns + ' beta'] = res.params[1]

            
        ff_report.loc[portf, 'info_ratio'] = np.sqrt(12) * res.params['const'] / res.resid.std()
        ff_report.loc[portf, 'treynor_ratio'] = 12 * portfolio[portf].mean() / res.params[1]
        ff_report.loc[portf, 'R-squared'] = res.rsquared
        ff_report.loc[portf, 'Tracking Error'] = (res.resid.std()*np.sqrt(12))

        if resid:
            bm_residuals[portf] = res.resid
            
            
        
    if resid:
        return bm_residuals
        
    return ff_report
    

In [83]:
def portfolio_stats(asset): 
    data_1 = excess_returns.loc[:'2011'][asset]
    data_2 = excess_returns.loc['2012':][asset]
    data_3 = excess_returns[asset]
    
    performance_1 = performance_summary(data_1, annualization_factor)[['Mean', 'Volatility', 'Sharpe Ratio', 'Min', 'VaR (0.05)', 'Max Drawdown']]
    performance_2 = performance_summary(data_2, annualization_factor)[['Mean', 'Volatility', 'Sharpe Ratio', 'Min', 'VaR (0.05)', 'Max Drawdown']]
    performance_3 = performance_summary(data_3, annualization_factor)[['Mean', 'Volatility', 'Sharpe Ratio', 'Min', 'VaR (0.05)', 'Max Drawdown']]
    
    performance_1.index = ['Inception through 2011']
    performance_2.index = ['2012 - Present']
    performance_3.index = ['Inception - Present']
    
    
    reg_sub = pd.concat([performance_1,performance_2,performance_3])
    return reg_sub

portfolio_stats('GMWAX')

,Mean,Volatility,Sharpe Ratio,Min,VaR (0.05),Max Drawdown
Inception through 2011,-0.265291,0.131867,-2.011809,-0.193379,-0.083767,-0.988490
2012 - Present,-0.106196,0.111332,-0.953862,-0.115577,-0.072478,-0.818206
Inception - Present,-0.192155,0.124798,-1.539722,-0.193379,-0.078500,-0.996149


Low Tail Risk
- No does not vary too much but tail risk is especially low from 2012 compared to the initial period. 

In [84]:
def ts_regressions(asset): 
    data_1 = excess_returns.loc[:'2011']
    data_2 = excess_returns.loc['2012':]
    data_3 = excess_returns
    sub_1 = time_series_regression(data_1.loc[:,[asset]], data_1.loc[:,['SPY']], annualization_factor)
    sub_2 = time_series_regression(data_2.loc[:,[asset]], data_2.loc[:,['SPY']], annualization_factor)
    sub_3 = time_series_regression(data_3.loc[:,[asset]], data_3.loc[:,['SPY']], annualization_factor)
    
    sub_1.index = ['Inception through 2011']
    sub_2.index = ['2012 - Present']
    sub_3.index = ['Inception - Present']
    
    
    reg_sub = pd.concat([sub_1,sub_2,sub_3])
    return reg_sub
ts_regressions('GMWAX')


,alpha_hat,SPY beta,info_ratio,treynor_ratio,R-squared,Tracking Error
Inception through 2011,-0.094369,0.619535,-1.280785,-0.428211,0.687802,0.073680
2012 - Present,-0.095545,0.641153,-1.830870,-0.165633,0.780285,0.052186
Inception - Present,-0.093859,0.627293,-1.452333,-0.306324,0.731836,0.064626


- GMWAX has a moderate beta with the market but it remains stable across hte subsamples
- There is no alpha since it is negative

In [85]:
portfolio_stats('GMGEX')


,Mean,Volatility,Sharpe Ratio,Min,VaR (0.05),Max Drawdown
Inception through 2011,-0.315536,0.164479,-1.918397,-0.176925,-0.095833,-0.995288
2012 - Present,-0.148307,0.239316,-0.619714,-0.661188,-0.084806,-0.937019
Inception - Present,-0.238661,0.203459,-1.173016,-0.661188,-0.092832,-0.999424


In [86]:
ts_regressions('GMGEX')


,alpha_hat,SPY beta,info_ratio,treynor_ratio,R-squared,Tracking Error
Inception through 2011,-0.090772,0.814692,-1.137190,-0.387307,0.764487,0.079821
2012 - Present,-0.134610,0.824561,-0.662567,-0.179862,0.279303,0.203165
Inception - Present,-0.111706,0.810187,-0.746662,-0.294575,0.459311,0.149607


Differences: 
- The portfolio in the GMWAX has consistently higher mean returns compared to GMGEX.Both portfolios show negative mean returns across all periods, indicating losses, but GMWAX has relatively less negative performance.
- The Sharpe Ratio is more negative for GMGEX, indicating worse risk-adjusted returns compared to GMWAX. GMWAX performs better in terms of balancing risk and return, although still negative.
- GMWAX demonstrates smaller drawdowns, reflecting better capital preservation over time.
- GMGEX is a higher beta strategy than GMWAX with the higher SPY Beta. Furthermore, it has a less negative information ratio which indicates that GMGEX has better risk adjusted performances.
  

## Forecast Regressions

Part 1

In [92]:

SPY = excess_returns.loc[:,['SPY']]
signal_1 = ['SPX DVD YLD']
factor_1 = signals.loc[:, signal_1].shift(1).squeeze()

if isinstance(factor_1, pd.Series):
    factor_1 = factor_1.to_frame(name=signal_1[0])
signal_reg_1 = time_series_regression(SPY, factor_1,  annualization_factor)
signal_reg_1.index = ['DP']
signal_reg_1


signal_2 = ['SPX P/E']
factor_2 = signals.loc[:,signal_2].shift(1).squeeze()
if isinstance(factor_2, pd.Series):
    factor_2 = factor_2.to_frame(name=signal_2[0])
signal_reg_2 = time_series_regression(SPY, factor_2,  annualization_factor)
signal_reg_2.index = ['EP']
signal_reg_2


signal_3 = ['SPX DVD YLD','SPX P/E','TNote 10YR']
factor_3 = signals.loc[:,signal_3].shift(1)
if isinstance(factor_3, pd.Series):
    factor_3 = factor_3.to_frame(name=signal_3[0])
signal_reg_3 = time_series_regression(SPY, factor_3,  annualization_factor, multiple_factors = True)
signal_reg_3.index = ['DP,EP,US10Y']

display(signal_reg_1,signal_reg_2,signal_reg_3)

,alpha_hat,SPX DVD YLD beta,info_ratio,treynor_ratio,R-squared,Tracking Error
DP,-1.01369,0.039441,-6.296905,-3.972964,0.103726,0.160982


,alpha_hat,SPX P/E beta,info_ratio,treynor_ratio,R-squared,Tracking Error
EP,0.445584,-0.002529,2.687659,61.95338,0.049403,0.165789


,alpha_hat,SPX DVD YLD beta,SPX P/E beta,TNote 10YR beta,info_ratio,treynor_ratio,R-squared,Tracking Error
"DP,EP,US10Y",-0.203724,0.022279,0.000051,-0.010624,-1.327618,-7.033415,0.185629,0.153451


Part 2

In [133]:
DP_return = (signals.loc[:,'SPX DVD YLD'].shift(1).to_frame() * signal_reg_1['SPX DVD YLD beta'].values[0])+(signal_reg_1['alpha_hat']/annualization_factor).values[0]
DP_return = DP_return.rename(columns={'SPX DVD YLD':'Forecasted Return'}) * 100
DP_forecast_return = pd.DataFrame(DP_return['Forecasted Return']*excess_returns.loc[:,'SPY'], columns=DP_return.columns, index=DP_return.index)


EP_return = (signals.loc[:,'SPX P/E'].shift(1).to_frame() * signal_reg_2['SPX P/E beta'].values[0])+(signal_reg_2['alpha_hat']/annualization_factor).values[0]
EP_return = EP_return.rename(columns={'SPX P/E':'Forecasted Return'}) * 100
EP_forecast_return = pd.DataFrame(EP_return['Forecasted Return']*excess_returns.loc[:,'SPY'], columns=EP_return.columns, index=EP_return.index)


forecasted_rets = (np.array(signals.shift(1).loc[:,['SPX DVD YLD','SPX P/E','TNote 10YR']]) @ np.array(signal_reg_3.loc[:,['SPX DVD YLD beta',	'SPX P/E beta',	'TNote 10YR beta']].T))
multiple_factor_return = (pd.DataFrame(forecasted_rets,columns = ['Forecasted Return'],index= signals.index)) 
multiple_factor_return['Forecasted Return'] = (multiple_factor_return['Forecasted Return'] + float((signal_reg_3['alpha_hat']/annualization_factor).values[0]))*100
multiple_forecast_return = pd.DataFrame(multiple_factor_return['Forecasted Return'] *excess_returns.loc[:,'SPY'], columns=multiple_factor_return.columns, index=multiple_factor_return.index)



In [149]:
factor_reg

,0.010316
date,
1997-01-31,0.010316
1997-02-28,-0.042635
1997-03-31,-0.098941
1997-04-30,0.012038
1997-05-30,0.013807
...,...
2024-06-28,-0.018264
2024-07-31,-0.040731
2024-08-30,-0.027751


In [152]:
strategy = {'DP': DP_forecast_return.dropna(),
          'EP': EP_forecast_return.dropna(),
          'DP-EP-US10Y': multiple_forecast_return.dropna()
         }
factor = excess_returns.loc[:,'SPY']
total_strategy_summary = []

for key,value in strategy.items():
    strat = strategy[key]
    strat_summary = performance_summary(strat, annualization_factor)
    strat_summary.index = [key]
    strat_summary['Negative Risk Premium Months'] = len(strat[strat['Forecasted Return'] - rf['TBill 3M'] <0])
    strat_summary['Total Months'] = len(strat)
    factor_reg = factor[strat.index[0]:].squeeze()
    if isinstance(factor_reg, pd.Series):
        factor_reg = factor_reg.to_frame(name='SPY')

    if isinstance(strat, pd.Series):
        strat = strat.to_frame(name=strat[0])
        
    ts = time_series_regression(strat, factor_reg , annualization_factor, False)
    strat_summary['Market Beta'] = ts['SPY beta'].values
    strat_summary['Market Alpha'] = ts['alpha_hat'].values
    strat_summary['Market Information Ratio'] = ts['info_ratio'].values
    
    total_strategy_summary.append(strat_summary)
    
total_strategy_df = pd.concat(total_strategy_summary)
  
total_strategy_df.loc[:,['Mean','Volatility','Sharpe Ratio','Max Drawdown','Market Beta','Market Alpha','Market Information Ratio']]




/var/folders/qq/h84pzsw93xbd60q11ssc5v640000gn/T/ipykernel_2037/172855033.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  strat_summary['Negative Risk Premium Months'] = len(strat[strat['Forecasted Return'] - rf['TBill 3M'] <0])
/var/folders/qq/h84pzsw93xbd60q11ssc5v640000gn/T/ipykernel_2037/172855033.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  strat_summary['Negative Risk Premium Months'] = len(strat[strat['Forecasted Return'] - rf['TBill 3M'] <0])
/var/folders/qq/h84pzsw93xbd60q11ssc5v640000gn/T/ipykernel_2037/172855033.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  strat_summary['Negative Risk Premium Months'] = len(strat[strat['Forecasted Return'] - rf['TBill 3M'] <0])


,Mean,Volatility,Sharpe Ratio,Max Drawdown,Market Beta,Market Alpha,Market Information Ratio
DP,0.497869,0.425072,1.171260,-0.820349,-1.262714,0.302814,0.825430
EP,0.341268,0.311727,1.094766,-0.809706,-1.359324,0.131289,0.627706
DP-EP-US10Y,0.733980,0.484967,1.513463,-0.511400,-1.464473,0.507758,1.220132


Part 3 

In [154]:
market_summary = performance_summary(excess_returns.loc[:,['SPY']], annualization_factor)
gmo_summary = performance_summary(excess_returns.loc[:,['GMWAX']].dropna(), annualization_factor)
strat_var= pd.concat([total_strategy_df.loc[:,['VaR (0.05)']],market_summary.loc[:,['VaR (0.05)']],gmo_summary.loc[:,['VaR (0.05)']]])
strat_var

,VaR (0.05)
DP,-0.082819
EP,-0.070586
DP-EP-US10Y,-0.071534
SPY,-0.099500
GMWAX,-0.078500


In [155]:
strats = {'DP': DP_forecast_return.dropna(),
          'EP': EP_forecast_return.dropna(),
          'DP-EP-US10Y': multiple_forecast_return.dropna()
         }
strat_summary_0011 =[]
for k,v in strats.items():
    strat = (strats[k]['2000':'2011']['Forecasted Return']).to_frame('Forecasted Returns')
    perf_summary = performance_summary(strat, annualization_factor)
    perf_summary.index = [k]
    strat_summary_0011.append(perf_summary)
    

strat_summary_df_0011 = pd.concat(strat_summary_0011)
strat_summary_df_0011.loc[:,['Mean','Volatility','Sharpe Ratio','Max Drawdown']]

,Mean,Volatility,Sharpe Ratio,Max Drawdown
DP,0.689627,0.525591,1.312098,-0.820349
EP,0.461415,0.314172,1.468668,-0.195466
DP-EP-US10Y,0.997842,0.570101,1.750292,-0.511400


All dynamic portfolios outperform the risk free rate

In [156]:
neg_risk_premium = total_strategy_df.loc[:,['Negative Risk Premium Months','Total Months']]
neg_risk_premium['Negative Risk Premium Months (%)'] = neg_risk_premium['Negative Risk Premium Months'] *100/ neg_risk_premium['Total Months']
neg_risk_premium

,Negative Risk Premium Months,Total Months,Negative Risk Premium Months (%)
DP,188,334,56.287425
EP,198,334,59.281437
DP-EP-US10Y,148,334,44.311377


In [157]:
market_summary.loc[:,['Mean','Volatility','Sharpe Ratio','VaR (0.05)','Max Drawdown']]

,Mean,Volatility,Sharpe Ratio,VaR (0.05),Max Drawdown
SPY,-0.156699,0.170194,-0.920703,-0.0995,-0.99291


In [158]:
total_strategy_df.loc[:,['Mean','Volatility','Sharpe Ratio','VaR (0.05)','Max Drawdown','Market Beta','Market Alpha','Market Information Ratio']]

,Mean,Volatility,Sharpe Ratio,VaR (0.05),Max Drawdown,Market Beta,Market Alpha,Market Information Ratio
DP,0.497869,0.425072,1.171260,-0.082819,-0.820349,-1.262714,0.302814,0.825430
EP,0.341268,0.311727,1.094766,-0.070586,-0.809706,-1.359324,0.131289,0.627706
DP-EP-US10Y,0.733980,0.484967,1.513463,-0.071534,-0.511400,-1.464473,0.507758,1.220132


No. judging by the tail risk metrics, the portfolios did not take on extra risk. 

## OOS Forecasting

Part 1 

In [159]:
dp_ratio = signals[['SPX DVD YLD']]
ep_ratio = signals[['SPX P/E']]

# using the regression results forecast forward 

dp_beta = sub_3['SPX DVD YLD beta'].values[0]
dp_alpha = sub_3['alpha_hat'].values[0]
dp_oos_returns = dp_ratio.shift(1) * dp_beta + dp_alpha/12 
dp_oos_returns.columns = ['Forecasted Returns DP']
dp_oos_returns*100

,Forecasted Returns DP
date,
1996-12-31,NaN
1997-01-31,0.771796
1997-02-28,0.631009
1997-03-31,0.636541
1997-04-30,0.745427
...,...
2024-06-28,0.063152
2024-07-31,0.020775
2024-08-30,0.013005


In [161]:
def OOS_r2(df, factors, start):
    y = df['SPY']
    X = sm.add_constant(factors)

    forecast_err, null_err = [], []

    for i,j in enumerate(df.index):
        if i >= start:
            currX = X.iloc[:i]
            currY = y.iloc[:i]
            reg = sm.OLS(currY, currX, missing = 'drop').fit()
            null_forecast = currY.mean()
            reg_predict = reg.predict(X.iloc[[i]])
            actual = y.iloc[[i]]
            forecast_err.append(reg_predict - actual)
            null_err.append(null_forecast - actual)
            
    RSS = (np.array(forecast_err)**2).sum()
    TSS = (np.array(null_err)**2).sum()
    
    return ((1 - RSS/TSS),reg)

In [162]:
reg_ep = OOS_r2(spy, ep_ratio.shift(1), 60)
OOS_RSquared_ep = reg_ep[0]
OOS_r2_ep = pd.DataFrame([[OOS_RSquared_ep]], columns = ['OOS R-Squared'], index = ['EP'])
reg_ep_params = reg_ep[1]


reg_dp = OOS_r2(spy, dp_ratio.shift(1), 60)
OOS_RSquared_dp = reg_dp[0]
OOS_r2_dp = pd.DataFrame([[OOS_RSquared_dp]], columns = ['OOS R-Squared'], index = ['DP'])
reg_dp_params = reg_dp[1]


reg_factors = OOS_r2(spy, signals.shift(1), 60)
OOS_RSquared_factors = reg_factors[0]
OOS_r2_factors = pd.DataFrame([[OOS_RSquared_factors]], columns = ['OOS R-Squared'], index = ['All'])
reg_factors_params = reg_factors[1]

In [163]:
oos_r2_sum = pd.concat([OOS_r2_ep,OOS_r2_dp,OOS_r2_factors])
oos_r2_sum

,OOS R-Squared
EP,-0.013791
DP,-0.045636
All,-0.080918


This forecasting strategy produces a negative OOS r-squared, which indicates our strategy fits the data worse than a horizontal line given by the expanding mean of the sample.

Part 2

In [167]:
def OOS_strat(df, factors, start, weight):
    returns = []
    y = df['SPY']
    X = sm.add_constant(factors)

    for i,j in enumerate(df.index):
        if i >= start:
            currX = X.iloc[:i]
            currY = y.iloc[:i]
            reg = sm.OLS(currY, currX, missing = 'drop').fit()
            pred = reg.predict(X.iloc[[i]])
            w = pred * weight
            returns.append((df.iloc[i]['SPY'] * w)[0])

    df_strat = pd.DataFrame(data = returns, index = df.iloc[-(len(returns)):].index, columns = ['Strat Returns'])
    return df_strat

In [168]:
factor = signals.loc[:,'SPX P/E'].shift(1).to_frame()
fund_ret= excess_returns.loc[factor.index[0]:,['SPY']]
OOS_EP_predict = OOS_strat(fund_ret,factor, 60, 100).rename(columns={'Strat Returns':'EP_OOS_Returns'})
OOS_EP_predict

,EP_OOS_Returns
date,
2001-12-31,0.047018
2002-01-31,0.114765
2002-02-28,0.154909
2002-03-29,-0.076695
2002-04-30,0.388740
...,...
2024-06-28,0.045590
2024-07-31,0.110532
2024-08-30,0.077426


In [171]:
factor = signals.loc[:, 'SPX DVD YLD'].shift(1).to_frame()
fund_ret= excess_returns.loc[factor.index[0]:,['SPY']]
OOS_DP_predict = OOS_strat(fund_ret,factor, 60, 100).rename(columns={'Strat Returns':'DP_OOS_Returns'})
OOS_DP_predict

,DP_OOS_Returns
date,
2001-12-31,0.047716
2002-01-31,0.111751
2002-02-28,0.138064
2002-03-29,-0.056391
2002-04-30,0.296589
...,...
2024-06-28,0.056024
2024-07-31,0.130341
2024-08-30,0.089711


In [172]:
factor = signals.loc[:,['SPX DVD YLD','SPX P/E','TNote 10YR']].shift(1)
fund_ret= excess_returns.loc[factor.index[0]:,['SPY']]
OOS_all_predict = OOS_strat(fund_ret,factor, 60, 100).rename(columns={'Strat Returns':'All_OOS_Returns'})
OOS_all_predict

,All_OOS_Returns
date,
2001-12-31,0.028078
2002-01-31,0.077115
2002-02-28,0.084243
2002-03-29,-0.020603
2002-04-30,0.176626
...,...
2024-06-28,0.060257
2024-07-31,0.132448
2024-08-30,0.079974


In [179]:
oos_prediction_sum = pd.concat([OOS_DP_predict.T,OOS_EP_predict.T,OOS_all_predict.T])
oos_prediction_sum = oos_prediction_sum.T

strats = {'DP': OOS_DP_predict.dropna(),
          'EP': OOS_EP_predict.dropna(),
          'All': OOS_all_predict.dropna(),
          'SPY':excess_returns.loc[OOS_all_predict.index[0]:,['SPY']].rename(columns={'SPY':'SPY_OOS_Returns'}),
          'US3M':rf['TBill 3M'].to_frame('US3M_OOS_Returns')
         }
factor = excess_returns.loc[:,['SPY']]
strat_summary =[]
for k,v in strats.items():
    strat = strats[k]
    perf_summary = performance_summary(strat, annualization_factor)
    perf_summary['Negative Risk Premium Months'] = len(strat[strat[k+'_OOS_Returns'] - rf['TBill 3M'] <0])
    perf_summary['Total Months'] = len(strat)
    perf_summary.index = [k]
    reg = time_series_regression(strat, factor[strat.index[0]:].squeeze().to_frame('SPY'), False)
    perf_summary['Market Beta'] = reg['SPY beta'].values
    perf_summary['Market Alpha'] = reg['alpha_hat'].values
    perf_summary['Market Information Ratio'] = reg['info_ratio'].values
    strat_summary.append(perf_summary)
    

strat_summary_df = pd.concat(strat_summary)
strat_summary_df.loc[:,['Mean','Volatility','Sharpe Ratio','VaR (0.05)','Max Drawdown','Market Beta','Market Alpha','Market Information Ratio']]

/var/folders/qq/h84pzsw93xbd60q11ssc5v640000gn/T/ipykernel_2037/614038411.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  perf_summary['Negative Risk Premium Months'] = len(strat[strat[k+'_OOS_Returns'] - rf['TBill 3M'] <0])
/var/folders/qq/h84pzsw93xbd60q11ssc5v640000gn/T/ipykernel_2037/614038411.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  perf_summary['Negative Risk Premium Months'] = len(strat[strat[k+'_OOS_Returns'] - rf['TBill 3M'] <0])
/var/folders/qq/h84pzsw93xbd60q11ssc5v640000gn/T/ipykernel_2037/614038411.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  perf_summary['Negative Risk Premium Months'] = len(strat[strat[k+'_OOS_Returns'] - rf['TBill 3M'] <0])
/var/folders/qq/h84pzsw93xbd60q11ssc5v640000gn/T/ipykernel_2037/614038411.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  perf_summary['Negative Risk Premium Months'] = len(stra

,Mean,Volatility,Sharpe Ratio,VaR (0.05),Max Drawdown,Market Beta,Market Alpha,Market Information Ratio
DP,0.191463,0.262730,0.728743,-0.082852,-0.784477,-1.276312,0.0,0.493772
EP,0.172862,0.300741,0.574788,-0.109675,-0.928909,-1.551068,0.0,0.229019
All,0.228703,0.234955,0.973395,-0.054393,-0.679894,-0.359631,0.0,0.867416
SPY,-0.086737,0.161099,-0.538408,-0.092920,-0.916290,1.000000,-0.0,-0.886757
US3M,0.258761,0.072832,3.552851,0.000137,-0.000203,-0.180181,0.0,3.489589


OOS performs worse than expected with the lower mean, higher volatility and tail risk measures. 

Part 3 

In [181]:
oos_prediction_sum = pd.concat([OOS_DP_predict.T,OOS_EP_predict.T,OOS_all_predict.T])
oos_prediction_sum = oos_prediction_sum.T

strats = {'DP': OOS_DP_predict.dropna(),
          'EP': OOS_EP_predict.dropna(),
          'All': OOS_all_predict.dropna(),
          'US3M':rf['TBill 3M'].to_frame('US3M_OOS_Returns')
         }
factor = excess_returns.loc[:,['SPY']]['2000':'2011']
strat_summary =[]
for k,v in strats.items():
    strat = strats[k]['2000':'2011']
    perf_summary = performance_summary(strat, annualization_factor)
    perf_summary['Negative Risk Premium Months'] = len(strat[strat[k+'_OOS_Returns'] - rf['2000':'2011']['TBill 3M'] <0])
    perf_summary['Total Months'] = len(strat)
    perf_summary.index = [k]
    reg = time_series_regression(strat, factor[strat.index[0]:].squeeze().to_frame('SPY'), False)
    perf_summary['Market Beta'] = reg['SPY beta'].values
    perf_summary['Market Alpha'] = reg['alpha_hat'].values
    perf_summary['Market Information Ratio'] = reg['info_ratio'].values
    strat_summary.append(perf_summary)
    

strat_summary_df_0011 = pd.concat(strat_summary)
strat_summary_df_0011.loc[:,['Mean','Volatility','Sharpe Ratio','VaR (0.05)','Max Drawdown','Market Beta','Market Alpha','Market Information Ratio']]

/var/folders/qq/h84pzsw93xbd60q11ssc5v640000gn/T/ipykernel_2037/2360539660.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  perf_summary['Negative Risk Premium Months'] = len(strat[strat[k+'_OOS_Returns'] - rf['2000':'2011']['TBill 3M'] <0])
/var/folders/qq/h84pzsw93xbd60q11ssc5v640000gn/T/ipykernel_2037/2360539660.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  perf_summary['Negative Risk Premium Months'] = len(strat[strat[k+'_OOS_Returns'] - rf['2000':'2011']['TBill 3M'] <0])
/var/folders/qq/h84pzsw93xbd60q11ssc5v640000gn/T/ipykernel_2037/2360539660.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  perf_summary['Negative Risk Premium Months'] = len(strat[strat[k+'_OOS_Returns'] - rf['2000':'2011']['TBill 3M'] <0])


,Mean,Volatility,Sharpe Ratio,VaR (0.05),Max Drawdown,Market Beta,Market Alpha,Market Information Ratio
DP,0.229812,0.255884,0.898111,-0.073539,-0.513260,-1.097816,0.0,0.205782
EP,0.317140,0.319588,0.992341,-0.106697,-0.628056,-1.535478,0.0,0.247385
All,0.162310,0.257816,0.629557,-0.054211,-0.679894,-0.773181,0.0,0.117710
US3M,0.274505,0.069383,3.956389,0.000412,-0.000152,-0.165479,0.0,3.700338


In [182]:
neg_risk_premium = strat_summary_df.loc[:,['Negative Risk Premium Months','Total Months']]
neg_risk_premium['Negative Risk Premium Months (%)'] = neg_risk_premium['Negative Risk Premium Months'] *100/ neg_risk_premium['Total Months']
neg_risk_premium

,Negative Risk Premium Months,Total Months,Negative Risk Premium Months (%)
DP,166,275,60.363636
EP,172,275,62.545455
All,153,275,55.636364
SPY,179,275,65.090909
US3M,0,335,0.000000


Yes, it is riskier given the higher percentage of months with negative risk premiums.